In [298]:
import numpy as np
import pandas as pd

In [304]:
# Загружаем подготовленные датасеты с городскими и коммерческими маршрутами 
# (с полными списками остановок в обоих направлениях и номерами кластеров, к которым относятся остановки)
r_df = pd.read_csv('all_routes_info.csv', index_col=0)
all_routes_df = r_df[['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2']]

comm_df = pd.read_csv('commerce_stops.csv', index_col=0)

In [327]:
# Основной алгоритм поиска городских маршрутов, которые похожи на указанный коммерческий. 
# В его основе лежит поиск наибольшей общей подпоследовательности у двух маршрутов
# Для указанного коммерческого маршрута просматриваются все доступные городские и находятся все общие последовательности 
# перегонов между каждой парой таких маршрутов. Перегон считается общим, если, соответственно, 
# пары начальных и конечных остановок коммерческого и городского маршрутов находятся в одних кластерах.

# Следующие две функции являются вспомогательными к функции lcs, которая находит максимальную общую подпоследовательность 
def sequential_slice(iterable, length):
    pool = tuple(iterable)
    assert 0 < length <= len(pool)
    tails = (pool[s:] for s in range(length))
    return zip(*tails)

def sequence_in_list(sequence, lst):
    pool = tuple(sequence)
    return any((pool == s for s in sequential_slice(lst, len(pool))))

# Непосредственно, функция, которая ищет максимальную общую подпоследовательность двух переданных строк с маршрутами
def lcs(str1, str2):
    a = str1.split(',')
    b = str2.split(',')
    
    is_max = True
    if len(a) > len(b):
        a, b = b, a
    for l in reversed(range(1, len(a)+1)):
        seq = [subseq for subseq in sequential_slice(a, l) if sequence_in_list(subseq, b)]
        if seq:
            break
    return seq

# Функция, обобщающая предыдущую. Результатом ее работы является полный список всех общих перегонов
def create_list_of_subroutes(str1, str2):
    res = 1
    all_subroutes = []
    while len(str1) > 0 and len(str2) > 0 and res != 0:
        seq = lcs(str1, str2)
        # Если нет общих перегонов или остались только общие остановки, мы завершаем работу (больше нет общих перегонов)
        if len(seq) == 0 or len(seq[0]) == 1:
            res = 0
        # Иначе мы добавляем наибольший общий в итоговый список всех общих и удаляем из обоих маршрутов эту последовательность
        else:
            all_subroutes.append(seq[0])
            substr = ""
            for i in seq[0]:
                substr += "," + i
            # удаление общей подпоследовательности из маршрутов
            new_str1_array = str1.split(substr)
            if len(new_str1_array) == 1:
                str1 = str1.split(substr)[0]
            else:
                str1 = str1.split(substr)[0] + str1.split(substr)[1]
                
            new_str2_array = str2.split(substr) 
            if len(new_str2_array) == 1:
                str2 = str2.split(substr)[0]
            else:
                str2 = str2.split(substr)[0] + str2.split(substr)[1]
    amount_of_similar = 0 # Переменная, в которую мы запишем количество общих перегонов (чтобы в дальнешем сортировать)           
    for i in range(len(all_subroutes)):
        amount_of_similar += len(all_subroutes[i]) - 1
    return all_subroutes, amount_of_similar  

# Функция, которая возвращает number_of_similar городских маршрутов, которые больше всего похожи на наш
def most_similar(comm_route, number_of_similar):
    comm_clusters1 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction1'].values[0]
    comm_clusters2 = comm_df[comm_df['route_number'] == comm_route]['clusters_in_direction2'].values[0]
    
    # Создаем zip с парами номер маршрута - тип транспорта для использования в алгоритме
    all_routes_names = all_routes_df['route_number'].values
    all_routes_types = all_routes_df['id_transport'].values
    all_routes = zip(all_routes_names, all_routes_types)
    
    number_of_stops = len(comm_clusters1.split(','))
    choosen_direction = 1
    direction_of_result = 1
    
    similarity_arr = []
    max_subroute = []
    if comm_clusters1 != '-1':
        for route_name, route_type in all_routes:
            all_routes_this_type = all_routes_df[all_routes_df['id_transport'] == route_type]
            current_clusters_1 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction1'].values[0]
            current_clusters_2 = all_routes_this_type[all_routes_this_type['route_number'] == route_name]['clusters_in_direction2'].values[0]
            subroutes1, amount1 = create_list_of_subroutes(comm_clusters1, current_clusters_1)
            subroutes2, amount2 = create_list_of_subroutes(comm_clusters1, current_clusters_2)
            if amount2 > amount1:
                subroutes = subroutes2
                amount = amount2
            else:
                subroutes = subroutes1
                amount = amount1
            if len(subroutes) == 0:
                similarity_arr.append([(route_name, route_type), 0, []])
            else:                       
                similarity_arr.append([(route_name, route_type), amount, subroutes])
    #print(similarity_arr)
    most_similar = sorted(similarity_arr, key=lambda x: x[1], reverse=True)[:number_of_similar]
    return most_similar, number_of_stops, choosen_direction, direction_of_result

# Функция, которая использует в себе указанные выше и выдает отчет о наиболее похожих маршрутах
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')
def find_most_similar_routes(comm_route, threshold, type_of_result):
    if type_of_result == "percent":
        number_of_similar = 5
    else:
        number_of_similar = threshold
    similar, number_of_stops,  choosen_direction, direction_of_result = most_similar(comm_route, number_of_similar)
    print('-------------------------------------')
    print('Для коммерческого маршрута № '+ str(comm_route) + ' найдено ' + str(number_of_similar) + ' наиболее похожих:')
    for route in similar:
        route_pair = route[0]
        route_name = route_pair[0]
        route_type = route_pair[1]
        similarity_percent = round(route[1] * 100 / (number_of_stops - 1))
        if type_of_result == "percent":
            if similarity_percent < threshold:
                break
        type_string = ""
        if route_type == 1:
            type_string = "Автобусный "
        elif route_type == 2:
            type_string = "Трамвайный "
        else:
            type_string = "Троллебусный "
        print(type_string + "маршрут № " + route_name + ", процент совпадения: ", similarity_percent, "%")
        print("На данном маршруте совпадают следующие перегоны (пока в виде id кластеров): ", route[2])
        print()

In [331]:
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')

# Пример использования в варианте 2
find_most_similar_routes(191, 5, 'number')

In [330]:
# Данная функция может использоваться в двух вариантах: 
# 1 - выдавать информацию о маршрутах, схожесть с которыми не менее threshold процентов (при использовании с флагом 'percent')
# 2 -  выдавать информацию о threshold наиболее похожих маршрутах (при использовании с флагом 'number')

# Пример использования в варианте 1
find_most_similar_routes(68, 90, 'percent')